## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
features = df.drop(['Страховые выплаты'], axis = 1)
target = df['Страховые выплаты']

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


$$
t = (X^TX^){-1}(x^T)y
$$

$$
t = ((XZ)^T(XZ))^{-1}(XZ)^Ty
$$

$$
t = (Z^TX^T(XZ))^{-1}(XZ)^Ty
$$

$$
t = (Z^T(X^TX)Z)^{-1}(XZ)^Ty
$$

$$
t = Z^{-1}(X^TX)^{-1}(Z^T)^{-1}Z^TX^Ty
$$

$$
t = Z^{-1}(X^TX)^{-1}[(Z^T)^{-1}Z^T]X^Ty
$$

$$
t = Z^{-1}[(X^TX)^{-1}X^Ty]
$$

$$
t = Z^{-1}w
$$

Найдем новые предсказания, назовем их z
$$
z = XZt = XZwZ^{-1} = Xw = a
$$

##### Таким образом мы доказали что предсказания не изменились, а векторы весов относятся как:
$$
w1 = wZ^{-1}
$$
где Z - обратимая матрица

## 3. Алгоритм преобразования

##### Т.к. Z - обратимая матрица, а значит квадратная, и на нее мы домножаем наши признаки, то размер Z должен быть 4х4

In [5]:
Z = np.random.randint(100, size = (4, 4))

##### Проверим матрицу Z на обратимость

In [6]:
Z_inv = np.linalg.inv(Z)

In [7]:
Z_inv

array([[-0.0291275 ,  0.00585557,  0.02140331,  0.00433951],
       [ 0.02717782, -0.01316458, -0.01791227,  0.01072203],
       [-0.00347559,  0.01335246, -0.0056469 ,  0.00317429],
       [ 0.01426318, -0.00183707,  0.00058169, -0.0086363 ]])

##### Проведем скалирование при знаков и рассчитаем векторы предсказаний

## 4. Проверка алгоритма

In [8]:
scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

##### Рассчитаем оба вектора весов и предсказаний и посмотрим на сумманую разницу - если она будет близка к нулю, значит мы сделали все как надо

In [9]:
w = np.linalg.inv(features_scaled.T.dot(features_scaled)).dot(features_scaled.T).dot(target)
predictions_1 = features_scaled @ w
features_scaled_2 = features_scaled @ Z
w1 = np.linalg.inv(features_scaled_2.T.dot(features_scaled_2)).dot(features_scaled_2.T).dot(target)
predictions_2 = features_scaled_2 @ w1

In [10]:
(predictions_1 - predictions_2).sum()

5.5398394205319335e-14

##### Разница крайне мала, а значит преобразование не приводит к ухудшению качества модели

In [11]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('R2_Score на исходных данных: ',r2_score(target, predictions))

R2_Score на исходных данных:  0.42494550286668


In [12]:
features_matrix = features_scaled @ Z
model.fit(features_matrix, target)
predictions_matrix = model.predict(features_matrix)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions_matrix))

R2_Score для преобразованных  признаков:  0.42494550286668


##### Результаты так же одинаковые, а значит преобразование защитило данные пользователей, и не ухудшило качество модели